In [1]:
# standard import 
import os
import sys
import tempfile

# OCC imports
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.BRepTools import breptools_Write
from OCC.Core.gp import gp_Vec
from OCC.Extend.ShapeFactory import translate_shp
from OCC.Extend.DataExchange import read_step_file, read_stl_file

In [2]:
# gmsh binary location
GMSH_BINARY = '/usr/local/gmsh/bin/gmsh'
assert os.path.isfile(GMSH_BINARY)
# create a temporary directory to store gmsh files
tmp  =  tempfile.TemporaryDirectory()
TMP_DIR = tmp.name
print("Files will be saved to ", TMP_DIR)

In [3]:
ventilator_shp = read_step_file(os.path.join('..', 'assets', 'models', 'Ventilator.stp'))

In [4]:
# dump the geometry to a brep file, check it worked
BREP_FILE_NAME = os.path.join(TMP_DIR, "ventilator.brep")
breptools_Write(ventilator_shp, BREP_FILE_NAME)
assert os.path.isfile(BREP_FILE_NAME)

# create the gmesh file
gmsh_file_content = """SetFactory("OpenCASCADE");

Mesh.CharacteristicLengthMin = 1;
Mesh.CharacteristicLengthMax = 5;

a() = ShapeFromFile(%s);
""" % BREP_FILE_NAME
GEO_FILENAME = os.path.join(TMP_DIR, "ventilator.geo")
gmsh_file = open(GEO_FILENAME, "w")
gmsh_file.write(gmsh_file_content)
gmsh_file.close()
assert os.path.isfile(GEO_FILENAME)

# call gmsh, generate an STL file
STL_FILENAME = os.path.join(TMP_DIR, "ventilator.stl")
os.system("%s %s -2 -o %s -format stl" % (GMSH_BINARY, GEO_FILENAME, STL_FILENAME))
assert os.path.isfile(STL_FILENAME)

import glob
print(glob.glob(os.path.join(TMP_DIR, '*')))

# load the stl file
meshed_ventilator_shp = read_stl_file(STL_FILENAME)

In [5]:
my_renderer = JupyterRenderer(size=(900, 900))

In [6]:
my_renderer.DisplayShape(translate_shp(ventilator_shp, gp_Vec(-100,0,0)), render_edges=True, shape_color="cyan")
my_renderer.DisplayShape(meshed_ventilator_shp, render_edges=True, shape_color="cyan")

In [7]:
my_renderer